# Strategy two:

In [ ]:
""" Loading packages """
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn as skl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from matplotlib.pyplot import figure
import scipy.stats as ss

%matplotlib inline
sns.set_style('whitegrid')

import math
import random
import pyomo.opt
from pyomo.core import Var
from pyomo.environ  import *

import time

""" Loading Battery Class """
from battery import Battery

## Data Preparation
Here:
* We read the data
* Fix the date to have a **Date format** which is DD/MM/YY HH:MM
* Check if there are missing values and fill them up (incase)
* Set 'timesteps' or 'Datetime' to be the index

In [ ]:
""" Read the data """

df = pd.read_csv("load_and_dam_data.csv")
df.columns = ['Datetime', 'load_kW', 'dam_price']
df['Datetime'] = [datetime.strptime(x, '%d/%m/%y %H:%M') for x in df['Datetime']]
pd.set_option('display.max_rows', None)

# filling data using the median function
def fillmissingvalues(data):
    median = data.median()
    data.fillna(median, inplace = True)
    
    return data 
#setting the datetime as an index
df = fillmissingvalues(df)
dataset = df
dataset = df.set_index("Datetime")
dataset.index = pd.to_datetime(dataset.index)
print(dataset.info())

### Series of functions incase they're needed to be used:
* A function that resamples the data
* A function that normalizes the data

In [ ]:
""" Function to resample the data """

def reSampleByValue(data, dateValue = 'H'):
    if (dateValue == 'D') or (dateValue == 'W') or (dateValue == 'M') or (dateValue == 'Y'):
    # Date Related like D: Daily, M: Monthly, Y: Yearly
        resampled_data = data.iloc[:,:].resample(dateValue).sum()
        
    elif (dateValue == 'H') or (dateValue == '15Min'):
    # Time Related like hourly and by 15 minutes
        if (dateValue == 'H'):
            # every hour
            by_hour = data.iloc[:,:].resample('H').sum()
            resampled_data = by_hour.groupby(by_hour.index).sum()
        elif (dateValue == '15Min'):
            #every 15 minutes
            resampled_data = data.iloc[:,:]
        
    else:
            resampled_data = 'Sorry, the values you can use are: D, W, M, Y, H and 15Min'
    
    # returns the data the way you chose to represent it (by hour, day, week, etc...)
    return resampled_data


dataset_day = reSampleByValue(dataset, dateValue = 'D')
dataset_hour = reSampleByValue(dataset, dateValue = 'H')


""" normalizing the data """

def theDataNormalizer(data):
    scaler = MinMaxScaler(feature_range = (0, 1))
    data_scaled = scaler.fit_transform(data)
    
    return data_scaled

#test
data_norm = theDataNormalizer(dataset.iloc[:,:])

Which day is going to be plotted and analysed? 

In [ ]:
d=21 #day number

# Step One: Day ahead bid commitment:

In [ ]:
#random uniform error between -1 and 1
#dataset['randError'] = ss.uniform.rvs(size=2977, loc =-1, scale=2)
#random normal distribution error between -1 and 1
#dataset['randError'] = ss.norm.rvs(size=2977, loc = 0, scale = 1/3, random_state = 1995) # without seed
dataset['randError'] = ss.gamma.rvs(a = 2, size=2977, loc = 0, scale=0.17, random_state= 1995)


# Synthetic Day-ahead Forecast 
dataset['SyntheticLoadForecast'] = dataset['load_kW'] + dataset['randError']

In [ ]:
# creating 10 scenarios for the forecasted deviations
num_timesteps = 96
num_scenarios = 50
devs = []
for i in range(num_scenarios):
        #devs.append(ss.uniform.rvs(size=num_timesteps, loc =-1, scale=2)) # Uniform
        #devs.append(ss.norm.rvs(size=num_timesteps, loc = 0, scale = 1/3)) # normal
        devs.append(ss.gamma.rvs(a = 2, size=num_timesteps, loc = 0, scale=0.17)) # logistic 


dev_table = pd.DataFrame(devs).transpose()

### Boolean Character Parameter for Day-ahead
I predefine the parameter to strictly reduce the imbalanced power going through every timestep t and scenario s:
* if deviation >= 0 then bool_char = 1
* else bool_char = 0

In [ ]:
# boolean characters: either charging or discharging
bool_char = [0 for y in range(num_timesteps*num_scenarios)]
bool_char = np.array(bool_char).reshape(num_timesteps,num_scenarios)
bool_char_columns = []
for i in range(1,num_scenarios+1):
    bool_char_columns.append(i)

bool_char = pd.DataFrame(bool_char, columns = bool_char_columns)
for t in range(num_timesteps):
    for s in range(num_scenarios):
        if(dev_table.iloc[t,s] >= 0):
            bool_char.iloc[t,s] = 1 
        else:
            bool_char.iloc[t,s] = 0

### Day Index column

Here an extra column is added to differentiate days between eachother.
There exist 31 days with the current dataset and each day has 96 slots so each day will have like:

Day 1: t=1 dayIndex = 1,
       t=2 dayIndex = 1,
       ...
       t=96 dayIndex = 1
...
...

Day 20: t=1 dayIndex = 20,
        t=2 dayIndex = 20,
        ...
        t=96 dayIndex = 20
...
...

In [ ]:
dataset_day = reSampleByValue(dataset, dateValue = 'D')
dataset['dayIndex'] = None
dayIndex = []
for i in range(1, len(dataset_day)+1): 
    for j in range(96):
        dayIndex.append(i)
        
for a in range(len(dataset)):
    dataset.iloc[a,4] = dayIndex[a]

Given that there:

* A synthetic forecast of the load
* The parameters of the uniformly randomized error (load forecast error) 
* The day ahead market price (Assumed to have this)

The point is to find an optimal way to choose a day-ahead market commitment which minimizes the expectation of the energy cost.

In [ ]:
# making np arrays (it is unnecessary but just makes my life easier)
p_forecast = dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values
dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price.values
deviation = dev_table
        
# Duration of a market dispatch time interval
MarketTime = 1
# Imbalance cost
imb_cost = 0.02
prob_s = 1/num_scenarios

#creating dummy data for battery
b_data = {'init_SOC': [0.6], # initial State of Charge
          'capacity':  [50.0], # Battery capacity [kWh]
          'char_P_Limit': [1.0], # Max charging power [kW]
          'dis_P_Limit': [1.0], # Max discharging power [kW]
          'char_P_Eff': [0.95], # charging power efficiency
          'dis_P_Eff': [0.95], # discharging power efficiency
          'batt_min_SOC':[0] # minimum SOC the battery should reach
          }

batteryData = pd.DataFrame (b_data, columns = ['capacity','init_SOC','char_P_Limit','dis_P_Limit',
                                               'char_P_Eff', 'dis_P_Eff', 'batt_min_SOC'])

# use battery class to store information
batt = Battery(state_of_charge = batteryData.loc[:,'init_SOC'][0],
           capacity = batteryData.loc[:,'capacity'][0],
           charging_power_limit = batteryData.loc[:,'char_P_Limit'][0],
           discharging_power_limit = batteryData.loc[:,'dis_P_Limit'][0],
           charging_efficiency = batteryData.loc[:,'char_P_Eff'][0],
           discharging_efficiency = batteryData.loc[:,'dis_P_Eff'][0],
           min_SOC = batteryData.loc[:,'batt_min_SOC'][0])

### Battery Losses

When power enters and exits the battery, it looses part of it due to the efficiency of the battery while charging and discharging.
Here the charging and discharging efficiency is 95% so 5% of the power is lost. In the below cell, the losses over all possible devaition scenarios are calculated.

Then later added to the objective function.

In [ ]:
batloss = []
for t in range(num_timesteps):
    for s in range(num_scenarios):
        batloss.append((1-batt.discharging_efficiency)*dev_table.iloc[t,s])
batloss = np.array(batloss).reshape(num_timesteps,num_scenarios)
batloss = pd.DataFrame(batloss, columns = bool_char_columns)

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" --- Model --- """""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""
m = ConcreteModel()

""" Sets of periods """
m.t = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1)
m.t_restrict = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-2)
m.t_beg = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1, 95)

""" Set of Batteries """
m.b = RangeSet(0, len(batteryData)-1)

""" Set of deviation scenarios """
m.s = RangeSet(0, len(deviation.columns)-1)


""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Variables """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""

#Here, the variables with their bounds that the model will use are defined. 
#These are what the model will try to optimize at every timestep

### The committed power constraint:
m.p_committed = Var(m.t)

# boolean character: either charging or discharging 
#m.bool_char = Var(m.t * m.b * m.s, within = Binary)

m.powercharge = Var(m.t * m.b * m.s, bounds=(0, batt.charging_power_limit), within=NonNegativeReals)
m.powerdischarge = Var(m.t * m.b * m.s, bounds=(0,batt.discharging_power_limit), within=NonNegativeReals)
m.b_energy = Var(m.t * m.b * m.s, bounds=(0, batt.capacity))
m.batterypower = Var(m.t * m.b * m.s, bounds=(-batt.discharging_power_limit,batt.discharging_power_limit))
m.SOC = Var(m.t * m.b * m.s, bounds=(0, 1))
m.p = Var(m.t * m.b * m.s, bounds=(0,99999))
m.q = Var(m.t * m.b * m.s, bounds=(0,99999))

#m.imbalance = Var(m.t * m.b * m.s)

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""" Day Ahead Objective Function """""""""
""""""""""""""""""""""""""""""""""""""""""""""""
def obj_dayahead_energy_cost(m):
    return sum(sum(prob_s * sum(((dam_price[t] * (p_forecast[t] + m.batterypower[t,b,s] - dev_table.iloc[t,s]) + batloss.iloc[t,s]) + (imb_cost * m.p[t,b,s]) + (imb_cost * m.q[t,b,s])) * MarketTime for t in m.t) for b in m.b) for s in m.s)


m.dayahead_total_energy_cost = Objective(rule=obj_dayahead_energy_cost,sense=minimize)

In [ ]:
""" All battery constraint """

#battery power constraint
def battery_power_constraint(m,t,b,s):
    return m.batterypower[t,b,s] == (m.powercharge[t,b,s] - m.powerdischarge[t,b,s])
# Battery energy at time t and scenario s
def b_energy_t_constraint(m,t,b,s):
    return m.b_energy[t,b,s] == m.SOC[t,b,s]*batt.capacity
# boolean constraints
def bool_control_charging(m,t,b,s):
    return ((m.powercharge[t,b,s]) <= (batt.charging_power_limit * bool_char.iloc[t,s]))
def bool_control_discharge(m,t,b,s):
    return ((m.powerdischarge[t,b,s]) <= (batt.discharging_power_limit * (1 - bool_char.iloc[t,s])))
# SOC Evolution Constraint
def soc_evolution(m, t_r, b,s):
    return m.b_energy[t_r+1,b,s] == m.b_energy[t_r,b,s] + (MarketTime*m.powercharge[t_r,b,s] * batt.charging_efficiency - (MarketTime*m.powerdischarge[t_r,b,s] / batt.discharging_efficiency))
# SOC initial state value at start of every day
def init_state_day_SOC(m,t,b,s):
    return m.b_energy[t, b,s] == batt.state_of_charge*batt.capacity
# minimum battery SOC constraint
def min_bat_soc(m,t,b,s):
    return m.b_energy[t,b,s] >= batt.min_SOC * batt.capacity
# Power committed is zero or larger at every time t
def p_comLargerthanzero(m,t):
    return m.p_committed[t] >= 0
def abs_constraint(m,t,b,s):
    return p_forecast[t] - dev_table.iloc[t,s] + m.batterypower[t,b,s] - m.p_committed[t] + batloss.iloc[t,s] + m.p[t,b,s] - m.q[t,b,s] == 0

m.init_state_day_SOC = Constraint(m.t_beg, m.b, m.s, rule=init_state_day_SOC)
#m.min_SOC_const = Constraint(m.t, m.b, rule=min_bat_soc)
m.p_com = Constraint(m.t, rule=p_comLargerthanzero)
m.battery_power = Constraint(m.t, m.b, m.s, rule=battery_power_constraint)
m.b_energy_constraint = Constraint(m.t, m.b, m.s, rule=b_energy_t_constraint)
m.Batt_boolean_charge = Constraint(m.t, m.b, m.s, rule=bool_control_charging)
m.Batt_boolean_discharge = Constraint(m.t, m.b, m.s, rule=bool_control_discharge)
m.soc_evol = Constraint(m.t_restrict, m.b, m.s, rule=soc_evolution)
m.abs_constraint = Constraint(m.t, m.b, m.s, rule = abs_constraint)

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Solver to be used """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
# set the path to the solver
opt = SolverFactory("cbc", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/cbc")

""""""""""""""""""""""""""""""""""""""""""
""" Solving the optimization problem: """
""""""""""""""""""""""""""""""""""""""""""

""""""""""""""" Start Timer """""""""""""""
total_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

results = opt.solve(m, tee=True)

""""""""""""""" End Timer """""""""""""""
elapsed = time.time() - total_start_time
print ('Time elapsed', elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
ind_DA = list(m.t)
p_committed = list(m.p_committed.get_values().values())

res_pyomo_DA = pd.DataFrame({'timesteps': ind_DA,'p_committed':p_committed,})
res_pyomo_DA = res_pyomo_DA.set_index("timesteps")

pd.set_option('display.max_rows', False)
res_pyomo_DA

In [ ]:
# saving the results in a csv file
result_py = np.asarray(res_pyomo_DA)
#np.savetxt('Results/StrategyTwo/Day-ahead/Day{0}/power_committed_Day{0}.csv'.format(d), result_py, delimiter=',')

### Graph of committed power vs forecasted power vs DAM prices at every time t for a single day

In [ ]:
resdata = {'forenorm': dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast,
        'damnorm': dataset.loc[dataset['dayIndex'] == d].dam_price,
        'comnorm': res_pyomo_DA.p_committed.values,
        }

dfres = pd.DataFrame (resdata, columns = ['forenorm','damnorm','comnorm'])

#normalized
dfnorm = theDataNormalizer(dfres)

Committed power vs forecasted power vs DAM prices at every time t for a single day

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()
lns1 = ax1.plot(res_pyomo_DA.index, dfres.iloc[:,0], "-s", color='#4daf4a',
                linewidth=3, label = 'forecasted power') # forecasted power
lns2 = ax1.plot(res_pyomo_DA.index, dfres.iloc[:,2], color ='#377eb8',
                linewidth=3, linestyle="-", label = 'committed power') # committed power
lns3 = ax2.plot(res_pyomo_DA.index, dfres.iloc[:,1], color = '#ff7f00',
                linewidth=3, linestyle= (0, (5, 3)), label = 'DAM prices') # DAM prices

# added these three lines
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 9, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('power (kW)', fontsize = 20)
ax2.set_ylabel('DAM prices', fontsize = 20)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Norm_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Uniform_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/gamma_committed_vs_forecast_vs_DAM_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

### Residuals plotting
Residuals = forecasted power - committed power

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))

ax1.plot(res_pyomo_DA.index, dfres.iloc[:,0] - dfres.iloc[:,2], "-H", color='black', linewidth=3, label = 'residuals') # residuals


ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('residuals', fontsize = 20)

#defining display layout 
plt.tight_layout()

# legend
ax1.legend(loc = 0, prop={'size': 18})


# Save figure
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Norm_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Uniform_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/gamma_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

### Residuals distribution

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sns.distplot(dfres.iloc[:,0] - dfres.iloc[:,2], color="dodgerblue", label="Residuals Distribution",  **kwargs)
plt.legend(loc = 0, prop={'size': 18});

# Save figure
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Norm_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/Uniform_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Day-ahead/Day{0}/gamma_dist_DAresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# Step Two: Battery Operation:
In this stage, it's assumed that the forecast is available and the day has arrived. Here we have the "commited power" which is the power that has been generated in the bidding stage. There is the "actual power" which is the "actual load power" and the "batter power". The difference between actual and commited power is named as "imbalance".

The point here is to optimize the schedule of the battery usage in order to minimize the expectation of the energy cost.

First the data being used for the optimization of the battery usage.

First, the optimization problem will be written using **Pyomo** optimization package:
* Creating the Model model
* setting the indexes:
    * model.t is the set which contains all the timesteps
    * model.t_restrict is the set that the same m.t but it eliminates the last index
    * model.t_beg is the set that contains values from m.t but incremented by 96 (0, 96, 192,...)

Second, The variables with their bounds that the model will use are defined. These are what the model will try to optimize at every timestep

Then, what is defined:
* Objective function: minimize energy cost
* constraints:
    * battery_power_constraint: batterypower = powercharge - powerdischarge
    * imbalance_equation: imbalance = p_actualLoad + batterypower - p_commited
    * b_energy_t_constraint: battery energy at every time t (0 <= battery energy <= battery capacity)
    * bool_control_charging and bool_control_discharging: these constraints tell the battery that it can only charge OR discharge during one time period
    * soc_evolution: calculates the battery energy at the next timestep t where t_retsrict is being used
    * init_state_day_SOC: initial state of charge at start of every day (which is 60%) where t_beg is being used

In [ ]:
p_actualLoad = dataset.loc[dataset['dayIndex'] == d].load_kW.values
p_commit = res_pyomo_DA.iloc[:,0]
dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price.values

""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" --- Model --- """""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""
model = ConcreteModel()

""" Sets of periods """
model.t = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1)
model.t_restrict = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-2)
model.t_beg = RangeSet(0, len(dataset.loc[dataset['dayIndex'] == d])-1, 95)

""" Set of Batteries """
model.b = RangeSet(0,len(batteryData)-1)

# boolean characters for the battery operation of the next day: either charging or discharging
bool_char_RT = [0 for y in range(len(dataset.loc[dataset['dayIndex'] == d])*len(batteryData))]
bool_char_RT = pd.DataFrame(bool_char_RT, columns = [1])
for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    for b in range(len(batteryData)):
        if(p_actualLoad[t] <= p_commit[t]):
            bool_char_RT.iloc[t,b] = 1
        else:
            bool_char_RT.iloc[t,b] = 0
            
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Variables """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""

#Here, the variables with their bounds that the model will use are defined. These are what the model will try to optimize at every timestep

# Amount of electricity (dis)charged of battery at time t  (kW)
model.powercharge = Var(model.t * model.b, bounds=(0, batt.charging_power_limit), within=NonNegativeReals)
model.powerdischarge = Var(model.t * model.b, bounds=(0,batt.discharging_power_limit), within=NonNegativeReals)
model.batterypower = Var(model.t * model.b, bounds=(-batt.discharging_power_limit,batt.discharging_power_limit))

# Energy stored in battery at time t [kWh]
model.b_energy = Var(model.t * model.b, bounds=(0, batt.capacity))
model.SOC = Var(model.t * model.b, bounds=(0, 1))

# boolean character: either charging or discharging 
#model.bool_char = Var(model.t * model.b, within = Binary)

model.imbalance = Var(model.t)

model.p = Var(m.t * model.b, bounds=(0,99999))
model.q = Var(m.t * model.b, bounds=(0,99999))

""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Objective Function """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
def obj_energy_cost(m):
    return sum(sum(((dam_price[t] * p_actualLoad[t]) + (dam_price[t] * (m.batterypower[t,b])) + (imb_cost * m.p[t,b]) + (imb_cost * m.q[t,b]))*MarketTime for t in m.t) for b in m.b)  

model.total_energy_cost = Objective(rule=obj_energy_cost,sense=minimize)

""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Constraints """""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
#battery power constraint
def battery_power_constraint(model,t,b):
    return model.batterypower[t,b] == (model.powercharge[t,b] - model.powerdischarge[t,b])
#Absolute value constraint
def imbalance_equation(model, t ,b):
    return model.imbalance[t] == (p_actualLoad[t] - p_commit[t] + model.batterypower[t,b])
# Battery energy at time t
def b_energy_t_constraint(model,t,b):
    return model.b_energy[t,b] == model.SOC[t,b]*batt.capacity
# boolean constraints
def bool_control_charging(model,t,b):
    return ((model.powercharge[t,b]) <= (batt.charging_power_limit * bool_char_RT.iloc[t,b]))
def bool_control_discharge(model,t,b):
    return ((model.powerdischarge[t,b]) <= (batt.discharging_power_limit * (1 - bool_char_RT.iloc[t,b])))
# SOC Evolution Constraint
def soc_evolution(model, t_r, b):
    return model.b_energy[t_r+1,b] == model.b_energy[t_r,b] + (MarketTime*model.powercharge[t_r,b] * batt.charging_efficiency - (MarketTime*model.powerdischarge[t_r,b] / batt.discharging_efficiency))
# SOC initial state value at start of every day
def init_state_day_SOC(model,t,b):
    return model.b_energy[t, b] == batt.state_of_charge*batt.capacity
# minimum battery SOC constraint
def min_bat_soc(model, t, b):
    return model.b_energy[t, b] >= batt.min_SOC * batt.capacity
# Absolute function constraint
def absolute_constraint(model,t,b):
    return model.imbalance[t] + model.p[t,b] - model.q[t,b] == 0


    
model.battery_power = Constraint(model.t,model.b,rule=battery_power_constraint)
model.b_energy_constraint = Constraint(model.t,model.b,rule=b_energy_t_constraint)
model.imb_eq = Constraint(model.t,model.b,rule=imbalance_equation)
model.Batt_boolean_charge = Constraint(model.t, model.b, rule=bool_control_charging)
model.Batt_boolean_discharge = Constraint(model.t, model.b, rule=bool_control_discharge)
model.soc_evol = Constraint(model.t_restrict, model.b, rule=soc_evolution)
model.init_state_day_SOC = Constraint(model.t_beg, model.b, rule=init_state_day_SOC)
model.min_SOC_const = Constraint(model.t, model.b, rule=min_bat_soc)
model.abs_eq = Constraint(model.t, model.b, rule = absolute_constraint)


In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Solver to be used """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
# set the path to the solver
opt = SolverFactory("cbc", executable="/Users/alaakamalcheaib/Desktop/UPC STUFF/ampl.macos64/cbc")

""""""""""""""""""""""""""""""""""""""""""
""" Solving the optimization problem: """
""""""""""""""""""""""""""""""""""""""""""

""""""""""""""" Start Timer """""""""""""""
total_start_time = time.time()
""""""""""""""""""""""""""""""""""""""""""

results_BatOp = opt.solve(model, tee=False)

""""""""""""""" End Timer """""""""""""""
elapsed = time.time() - total_start_time
print ('Time elapsed', elapsed)
""""""""""""""""""""""""""""""""""""""""""

In [ ]:
ind_RT = list(model.t)
powerc_RT = list(model.powercharge.get_values().values())
powerd_RT = list(model.powerdischarge.get_values().values())
bpow_RT = list(model.batterypower.get_values().values())
bsoc_RT = list(model.SOC.get_values().values())
imb_RT = list(model.imbalance.get_values().values())
bEn_RT = list(model.b_energy.get_values().values())

res_pyomo_RT = pd.DataFrame(
    {'timesteps': ind_RT,
     'power_charge': powerc_RT,
     'power_discharge': powerd_RT,
     'battery_power': bpow_RT,
     'battery_SOC': bsoc_RT,
     'imbalance': imb_RT,
     'battery_energy': bEn_RT
    })
res_pyomo_RT = res_pyomo_RT.set_index("timesteps")

In [ ]:
#normalized
dfres_RT = pd.DataFrame(
    {'forenorm': dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast,
     'damnorm': dataset.loc[dataset['dayIndex'] == d].dam_price,
     'comnorm': res_pyomo_DA.p_committed.values,
     'loadnorm': dataset.loc[dataset['dayIndex'] == d].load_kW,
     'batSOCnorm': bsoc_RT
    })
dfnorm_RT = theDataNormalizer(dfres_RT)

Committed vs Actual load power vs battery SOC at every time t for a single day

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()

lns1 = ax1.plot(dfres_RT.index, dfres_RT.iloc[:,2], "-", color= '#377eb8',
                linewidth=3, label = 'committed power')
lns2 = ax1.plot(dfres_RT.index, dfres_RT.iloc[:,3], "-s", color= '#e41a1c',
                linewidth=3, label = 'actual load power')
lns3 = ax2.plot(dfres_RT.index, dfres_RT.iloc[:,4], color= '#a65628',
                linewidth=3, linestyle= (0, (5, 3)), label = 'battery SOC')


# added these three lines
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 9, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize=20)
ax1.set_ylabel('power (kW)', fontsize=20)
ax2.set_ylabel('Battery SOC', fontsize=20)

#defining display layout 
plt.tight_layout()
# Save figure
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Norm_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Uniform_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/gamma_forecast__committed_realLoad_BatterySOC_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# show graph
plt.show()

### Residuals
Residuals = committed - actual
Residuals at every time t for a single day

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax1.plot(res_pyomo_DA.index, dfres_RT.iloc[:,2] - dfres_RT.iloc[:,3], "-H", color='black', linewidth=3, label = 'residuals') # residuals

ax1.legend(loc = 0, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('residuals', fontsize = 20)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Norm_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Uniform_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/gamma_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

### Residual distribution

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sns.distplot(dfres_RT.iloc[:,3] - dfres_RT.iloc[:,2], color="dodgerblue", label="Residuals Distribution",  **kwargs)
plt.legend(loc = 0, prop={'size': 18});

# Save figure
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Norm_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Uniform_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/gamma_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

In [ ]:
pd.set_option('display.max_rows', False)
res_pyomo_RT

In [ ]:
imbalance_cost = res_pyomo_RT.iloc[:,4] * imb_cost

dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price
bidding_cost = []


for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    result = res_pyomo_DA.iloc[t,0] * dam_price.iloc[t]
    bidding_cost.append(result)

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()

lns1 = ax1.plot(dfres_RT.index, bidding_cost, "-", color= '#984ea3',linewidth=3, label = "bidding cost")
lns2 = ax2.plot(dfres_RT.index, imbalance_cost, "-s", color= '#f781bf',linewidth=3, label = "imbalance cost")

# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 3, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize=18)
ax1.set_ylabel('bidding costs', fontsize=18)
ax2.set_ylabel('Imbalances', fontsize=18)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Norm_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/Uniform_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyTwo/Battery-operation/Day{0}/gamma_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

bidding vs imbalance cost at every time t for a single day

In [ ]:
#Strategy Two
total_bidding_cost = 0
total_imbalance_cost = 0
total_overall_cost = 0


for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    total_bidding_cost += bidding_cost[t]
    total_imbalance_cost += abs(imbalance_cost[t])
    total_overall_cost += dam_price.iloc[t] * (p_commit[t] + res_pyomo_RT.iloc[t,4]) + abs(imbalance_cost[t])
    
total_overall_cost,total_bidding_cost,total_imbalance_cost